In [1]:
import os

import pandas as pd
import numpy as np

# Coppied from sample main
local_path = '/home/milosh-dr/code/MPs'
files = os.listdir(local_path)
# 
results = []
dfs = []
for filename in files:
    if filename.startswith('results'):
        results.append(filename)

for filename in sorted(results):
    df = pd.read_csv(filename)
    dfs.append(df)

Pierdolisz się jak idiota z uzupełnieniem wartości dla KO. Rozwiąż to jakoś sensownie durniu.

In [4]:
df = pd.concat(dfs, axis=1)

In [12]:
df.head()

,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6,62/7,62/8,...,51/24,51/25,51/26,51/27,51/28,51/29,51/30,51/31,51/32,51/33
0,PiS,Adamczyk Andrzej,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
1,PiS,Andruszkiewicz Adam,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
2,PiS,Andzel Waldemar,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
3,PiS,Ardanowski Jan Krzysztof,Przeciw,Za,Przeciw,Przeciw,Za,Nieobecny,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
4,PiS,Arent Iwona,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN


Getting rid of incorrectly parsed data

In [9]:
nans = df.isna().sum()
nans[nans>0]

59/244     24
51/33     460
dtype: int64

In [19]:
df.drop(['59/244', '51/33'], axis=1, inplace=True)

Coding string descriptions of votes into numeric values. Marking absences with np.nans for now

In [34]:
mapping = {}
values = df.iloc[:,5].value_counts().index.tolist()
new_values = [0, 1 , np.nan, .5]
for value, new_value in zip(values, new_values):
    mapping[value] = new_value
mapping

{'Przeciw': 0, 'Za': 1, 'Nieobecny': nan, 'Wstrzymał się': 0.5}

In [62]:
for col in df.columns[2:]:
    df[col] = df[col].map(mapping)

There might be some meaningful absences, when MPs actually expressed their disagreement, but one need to be very careful replacing those values. Let's explore this. 

In [71]:
def sum_nan(col):
    """Aggregation function that sums np.nans"""
    return col.isna().sum()

not_present = df.groupby('Party').agg(sum_nan)

In [83]:
party_sizes = df[['Party', 'MPS']].groupby('Party').agg(len)['MPS']
not_present['party_size'] = party_sizes

In [136]:
def absence_checker(row):
    """As a rule of thumb we decide to take a closer look whenever more then 75% of party members were absent.
    Considering only parties having more then 5 members"""
    
    if row['party_size'] < 6:
        row[:]=np.nan
        return row
    
    row[~(row>.75*row['party_size'])] = np.nan
    return row


meaningful_absence = not_present.apply(absence_checker, axis=1)
subset = meaningful_absence.columns[1:-1].tolist()
meaningful_absence = (meaningful_absence
                      .dropna(axis=0, how='all', subset=subset)
                      .dropna(axis=1, how='all')
                      .drop('party_size', axis=1))
meaningful_absence

,55/62
Party,
KO,109.0


Since there was only one situation revealing strong probability of meaningful absence I'll imput this with 0.

In case of this analysis it's better to assume that in general MPs that are absent on a particular vote normally would vote with majority of the party they belong to. It wouldn't be reasonable to imput anomalies ourselves if we search for those.

In [137]:
df

,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6,62/7,62/8,...,51/23,51/24,51/25,51/26,51/27,51/28,51/29,51/30,51/31,51/32
0,PiS,Adamczyk Andrzej,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
1,PiS,Andruszkiewicz Adam,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
2,PiS,Andzel Waldemar,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,PiS,Ardanowski Jan Krzysztof,0.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,PiS,Arent Iwona,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,PS,Ścigaj Agnieszka,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
456,niez.,Ajchler Zbigniew,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
457,niez.,Galla Ryszard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
458,niez.,Mejza Łukasz,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Pierdolisz się jak idiota z uzupełnieniem wartości dla KO. Rozwiąż to jakoś sensownie durniu.

In [143]:
tmp = df.copy()

In [161]:
tmp.loc[np.where(tmp['Party']=='KO'), '55/62']

228    NaN
229    0.0
230    NaN
231    0.5
232    NaN
      ... 
349    NaN
350    NaN
351    NaN
352    NaN
353    NaN
Name: 55/62, Length: 126, dtype: float64

In [158]:
tmp['55/62'].isna().sum()

126

In [155]:
tmp.loc[np.where(tmp['Party']=='KO'), '55/62'].fillna(0, inplace=True)

In [160]:
tmp.iloc[np.where(tmp['Party']=='KO')]['55/62'] = tmp.loc[np.where(tmp['Party']=='KO'), '55/62'].fillna(0)

/tmp/ipykernel_2293/2708035599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.iloc[np.where(tmp['Party']=='KO')]['55/62'] = tmp.loc[np.where(tmp['Party']=='KO'), '55/62'].fillna(0)


In [145]:
tmp.groupby('Party').apply(lambda x: x.fillna(round(x.mean())))

/tmp/ipykernel_2293/392941792.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  tmp.groupby('Party').apply(lambda x: x.fillna(round(x.mean())))
/tmp/ipykernel_2293/392941792.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  tmp.groupby('Party').apply(lambda x: x.fillna(round(x.mean())))


,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6,62/7,62/8,...,51/23,51/24,51/25,51/26,51/27,51/28,51/29,51/30,51/31,51/32
0,PiS,Adamczyk Andrzej,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
1,PiS,Andruszkiewicz Adam,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
2,PiS,Andzel Waldemar,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,PiS,Ardanowski Jan Krzysztof,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,PiS,Arent Iwona,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,PS,Ścigaj Agnieszka,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
456,niez.,Ajchler Zbigniew,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
457,niez.,Galla Ryszard,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
458,niez.,Mejza Łukasz,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
